<a href="https://colab.research.google.com/github/Haooo517/Generation_AI/blob/main/NTNU_41147006S_%E8%B3%87%E5%B7%A5115_%E5%BC%B5%E8%A8%93%E8%B1%AA_HW10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 製作屬於自己的圖片生成機器人

首先下載所有這次要用到的東西 <br>
為了更清楚地顯示所有要用到的套件 <br>
因此這邊把每一個都分開下載

In [ ]:
!pip install diffusers --upgrade
!pip install transformers --upgrade
!pip install accelerate --upgrade
!pip install gradio --upgrade
!pip install safetensors --upgrade
!pip install huggingface_hub --upgrade

再來是要先登入 Hugging Face <br>
這樣才能使用裡面提供的非開源模型 <br>
所以也要用到 Google Colab 的密鑰來存 Token

In [ ]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HuggingFace')
login(token=hf_token)

再來把等等會用到的套件都 import 進來 <br>

In [ ]:
import torch
import gradio as gr
from diffusers import StableDiffusionXLPipeline

接著設定好模型 <br>
並且讀到 Stable Diffusion 的 pipeline 裡面

In [ ]:
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

然後是設定這次的機器人 <br>
這次我給他的主題是動漫風機器人 <br>
他可以讓玩家自由選擇要用哪種動畫風格生成圖片 <br>
實現的方式就是先讓使用者輸入想要的圖片 <br>
接著選擇風格後點擊修飾 prompt 就能把 prompt 加上與該風格有關的文字 <br>
先設定不同風格對應到要添加的文字

In [ ]:
STYLE_SUFFIXES = {
    "日式動漫風": ", japanese anime style, vibrant colors, 2D illustration",
    "美式卡通風": ", cartoon style, flat shading, retro animation look",
    "電玩像素風": ", pixel art, 8-bit video game style, retro aesthetic",
    "Q版動畫風": ", chibi character, cute, large head, small body, colorful 2D style",
    "迪士尼電影風": ", disney style, 3D animation, cinematic lighting, expressive characters"
}

再來是點擊修飾按鈕時要觸發的函式 <br>
也就是將剛剛設定的文字加到使用者的輸入上

In [ ]:
def refine_prompt(user_prompt, style):
    suffix = STYLE_SUFFIXES.get(style, "")
    if not user_prompt.strip():
        return "Please enter a prompt first!"
    return user_prompt.strip() + suffix

接著是產生圖片的函式 <br>
由於我設計的介面可以讓使用者自由設定 guidance 和 steps <br>
以及決定隨機種子 <br>
所以帶入進來的參數會比較多

In [ ]:
def generate_image(prompt, style, guidance, steps, seed):
    suffix = STYLE_SUFFIXES.get(style, "")
    full_prompt = prompt + suffix if suffix not in prompt else prompt
    generator = torch.Generator("cuda").manual_seed(seed)
    image = pipe(prompt=full_prompt,
                 guidance_scale=guidance,
                 num_inference_steps=steps,
                 generator=generator).images[0]
    return image

最後就是用 Gradio 製作圖形化介面 <br>
並且生成幾張圖片來測試看看

In [5]:
with gr.Blocks(title="Cartoona - 多風格卡通圖片生成器") as demo:
    gr.Markdown("# 🎨 Cartoona - 多風格卡通圖片生成器")
    gr.Markdown("輸入提示詞 → 選擇風格 → 點「🪄 修飾提示詞」可自動補上風格描述 → 生成卡通風格圖片！")

    with gr.Row():
        prompt = gr.Textbox(label="提示詞", placeholder="請用英文輸入，例如：A cat walking in the city")
        style = gr.Dropdown(
            label="選擇風格",
            choices=list(STYLE_SUFFIXES.keys()),
            value="日式動漫風"
        )

    with gr.Row():
        refine_btn = gr.Button("🪄 修飾提示詞")
        refine_btn.click(fn=refine_prompt, inputs=[prompt, style], outputs=prompt)

    with gr.Row():
        guidance = gr.Slider(1, 20, value=7.5, label="🎨創意<--->切題🤖")
        steps = gr.Slider(10, 100, value=30, step=5, label="⏳效率<--->品質🖼️")
        seed = gr.Number(value=42, label="🎲隨機種子（相同隨機種子和輸入能產出相同結果）")

    submit = gr.Button("✨ 生成圖片")
    output = gr.Image(type="pil", label="生成圖片")

    submit.click(fn=generate_image,
                 inputs=[prompt, style, guidance, steps, seed],
                 outputs=output)

demo.launch(share=True)

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ddebf7b29a8d46268b.gradio.live
